In [84]:
import numpy as np
import matplotlib.pyplot as plt
from typing_extensions import Literal
from typing import Optional, List
from Layer import Layer
#from activation_functions import *


class MLP(object):
  # Multilayer Perceptron class

  def __init__(self, 
    layers: List[Layer],
    learning_rate: float = 0.1,
    a_linear = Optional[float], 
    b_linear = Optional[float]):

    self.layers = layers # List of layers
    self.learning_rate = learning_rate # Learning rate
    self.errors  = [] # List to store errors
  
  def forward(self, stimulus)->np.ndarray:
    '''
    Forward propagation
    '''
    for layer in self.layers:
      print("is")
      stimulus = layer.forward(stimulus)
    return stimulus


  def backward(self,  y: np.ndarray)->None:
    '''
    Backward propagation
    '''
    local_gradient = y - self.layers[-1].output
    for layer in reversed(self.layers):
      print("local_gradient", local_gradient.shape)
      local_gradient = layer.backward(local_gradient)

  def update_weights(self):
    '''
    Update weights
    '''
    for layer in self.layers:
      layer.update(self.learning_rate)

  def train(self, x: np.ndarray, y: np.ndarray, epochs: int):
    '''
    Train MLP
    '''
    for epoch in range(epochs):
      y_output = self.forward(x)
      error = abs(y - y_output)
      self.backward(y)
      self.update_weights()

      if epoch%100 == 0:
        print("Epoch: ", epoch, "Error: ", np.mean(error))
      self.errors.append(np.mean(error))

  def get_gradients(self):
    '''
    Get gradients
    '''
    gradients = []
    for layer in self.layers:
      gradients.append(layer.weights_grad)
    return gradients
    
  def predict(self, x):
    y_output = self.forward(x)
    return y_output

In [99]:
import numpy as np
from typing import Optional

class Layer:

    def __init__(
        self, 
        input_size: np.ndarray, 
        output_size: np.ndarray, 
        activation: str,
        a_linear: Optional[float] = 1.0,
        b_linear: Optional[float] = 0,
      ):
        self.weights = np.random.rand(output_size, input_size)

        self.stimulus = None
        self.local_field = None
        self.output = None

        self.local_gradient = None
        self.weights_grad = None

        self.activation = activation
        self.a_linear = a_linear
        self.b_linear = b_linear

    def forward(self, inputs: np.ndarray) -> np.ndarray:
        self.stimulus = inputs
        self.local_field = np.matmul(self.weights, self.stimulus)
        self.output = self.phi(self.local_field)

        return self.output

    def backward(self, previous_gradient: np.ndarray) -> np.ndarray:
        print("is", previous_gradient.shape, self.local_field.shape,self.compute_derivative(self.local_field).shape)
        self.local_gradient = np.multiply(
            previous_gradient, self.compute_derivative(self.local_field)
        )
        print("local_gradient shape", self.local_gradient.shape, self.stimulus.T.shape)
        self.weights_grad = np.matmul(self.local_gradient, self.stimulus.T)
        print("weights_grad shape", self.weights.shape,self.weights_grad.shape)
        self.weights_grad = np.matmul(self.weights.T, self.weights_grad)
        return self.local_gradient

    def compute_derivative(self, v:np.ndarray) -> np.ndarray:
        if(self.activation == "Linear"):
          return np.zeros(v.shape)+1
        elif(self.activation == "Sigmoid"):
            return self.phi(v)*\
            (1-self.phi(v))
        elif(self.activation == "Tanh"):
            return 1 - self.phi(v)**2

    def update(self, learning_rate: float) -> None:
        self.weights = self.weights - learning_rate * self.weights_grad

    def phi(self, v: np.ndarray) -> np.ndarray:
        if(self.activation == "Linear"):
          return self.a_linear*v + self.b_linear
        elif(self.activation == "Sigmoid"):
            return 1/(1+np.exp(-v))
        elif(self.activation == "Tanh"):
            return np.tanh(v)


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from typing_extensions import Literal
from typing import Optional, List
from sklearn import datasets

In [8]:
iris = datasets.load_iris()
X_iris = iris.data
y_iris = iris.target
X_iris.shape

(150, 4)

In [100]:
model = MLP(layers = [Layer(4, 2, activation = 'Sigmoid'),Layer(2, 1, activation = 'Sigmoid')], learning_rate = 0.1)
model.train(X_iris.T, y_iris.T, epochs = 1000)
grads = model.get_gradients()
print(len(grads))

is
is
local_gradient (1, 150)
is (1, 150) (1, 150) (1, 150)
local_gradient shape (1, 150) (150, 2)
weights_grad shape (1, 2) (1, 2)
local_gradient (2, 2)
is (2, 2) (2, 150) (2, 150)


ValueError: operands could not be broadcast together with shapes (2,2) (2,150) 

In [57]:
predictions = model.predict(X_iris.T)
error = np.mean(np.abs(predictions-y_iris.T))
error

0.8333333333333334